In [1]:
'''
Here are some preliminary imports. We'll need more.
Feel free to install and import any packages you'd like.
'''
import requests
import os
import json
import numpy as np
import pandas as pd

# Additional Libraries
import config #to pull API secret key

In [14]:
'''
Let's retrieve all issues in the Apache Airflow OSS project using the github api. 
Insert your username below (in two places).

You should end up with about 20k issues, including ones that date back to 2015 when the project was created.
'''
def get_issues_from_api():
    session = requests.Session()
    url = "https://api.github.com/repos/apache/airflow/issues?state=all"
    response = session.get(
        url,
        auth=(
            "AmirZahre", # your github username
            config.api_secret
        ),
    )
    yield response.json()

    page=2
    while 'next' in response.links:
        response = session.get(
            url,
            auth=(
                "AmirZahre", # your github username
                config.api_secret
            ),
            params={"page": page}
        )
        page += 1
        yield response.json()
        

def get_all_issues(cache_path="airflow_issues.feather"):
    if os.path.exists(cache_path):
        print("retrieving issues from cache")
        df = pd.read_feather(cache_path)
    else:
        print("retrieving issues from API")
        df_shards = []
        for i, page in enumerate(get_issues_from_api()):
            print(f"retrieving page {i}")
            if not page:
                print(f"No issues found on page {i}: {page}")
                continue
            page_df = pd.DataFrame(page)
            df_shards.append(page_df)
        df = pd.concat(df_shards).reset_index()
        print(f"Saving results to {cache_path}")
        df.to_feather(cache_path)
    return df


In [15]:
df = get_all_issues()

retrieving issues from cache


In [22]:
pd.set_option('display.max_columns',None)
df.head()

,index,url,repository_url,labels_url,comments_url,events_url,html_url,id,node_id,number,title,user,labels,state,locked,assignee,assignees,milestone,comments,created_at,updated_at,closed_at,author_association,active_lock_reason,draft,pull_request,body,reactions,timeline_url,performed_via_github_app
0,0,https://api.github.com/repos/apache/airflow/is...,https://api.github.com/repos/apache/airflow,https://api.github.com/repos/apache/airflow/is...,https://api.github.com/repos/apache/airflow/is...,https://api.github.com/repos/apache/airflow/is...,https://github.com/apache/airflow/pull/20767,1096939225,PR_kwDOAgUK284wsGiW,20767,Rename amazon EMR hook name,{'avatar_url': 'https://avatars.githubusercont...,"[{'color': 'f89500', 'default': False, 'descri...",closed,False,None,[],None,3,2022-01-08T14:15:31Z,2022-01-08T15:26:11Z,2022-01-08T15:26:09Z,CONTRIBUTOR,None,False,{'diff_url': 'https://github.com/apache/airflo...,Renamed emr hook from Elastic MapReduce to Ama...,"{'+1': 0, '-1': 0, 'confused': 0, 'eyes': 0, '...",https://api.github.com/repos/apache/airflow/is...,None
1,1,https://api.github.com/repos/apache/airflow/is...,https://api.github.com/repos/apache/airflow,https://api.github.com/repos/apache/airflow/is...,https://api.github.com/repos/apache/airflow/is...,https://api.github.com/repos/apache/airflow/is...,https://github.com/apache/airflow/issues/20766,1096938561,I_kwDOAgUK285BYfRB,20766,Status of testing Providers that were prepared...,{'avatar_url': 'https://avatars.githubusercont...,"[{'color': '5319e7', 'default': False, 'descri...",open,False,None,[],None,0,2022-01-08T14:12:48Z,2022-01-08T14:12:48Z,None,MEMBER,None,None,None,### Body\n\nI have a kind request for all the ...,"{'+1': 0, '-1': 0, 'confused': 0, 'eyes': 0, '...",https://api.github.com/repos/apache/airflow/is...,None
2,2,https://api.github.com/repos/apache/airflow/is...,https://api.github.com/repos/apache/airflow,https://api.github.com/repos/apache/airflow/is...,https://api.github.com/repos/apache/airflow/is...,https://api.github.com/repos/apache/airflow/is...,https://github.com/apache/airflow/pull/20765,1096903757,PR_kwDOAgUK284wsBH7,20765,Add documentation for an ad-hoc release of 2 p...,{'avatar_url': 'https://avatars.githubusercont...,"[{'color': 'd4c5f9', 'default': False, 'descri...",closed,False,None,[],None,1,2022-01-08T11:24:41Z,2022-01-08T13:59:11Z,2022-01-08T13:59:11Z,MEMBER,None,False,{'diff_url': 'https://github.com/apache/airflo...,<!--\r\nThank you for contributing! Please mak...,"{'+1': 0, '-1': 0, 'confused': 0, 'eyes': 0, '...",https://api.github.com/repos/apache/airflow/is...,None
3,3,https://api.github.com/repos/apache/airflow/is...,https://api.github.com/repos/apache/airflow,https://api.github.com/repos/apache/airflow/is...,https://api.github.com/repos/apache/airflow/is...,https://api.github.com/repos/apache/airflow/is...,https://github.com/apache/airflow/pull/20764,1096847036,PR_kwDOAgUK284wr3Zo,20764,Update metric name in documentation,{'avatar_url': 'https://avatars.githubusercont...,"[{'color': '0030b5', 'default': False, 'descri...",open,False,None,[],None,1,2022-01-08T06:15:11Z,2022-01-08T15:21:56Z,None,CONTRIBUTOR,None,False,{'diff_url': 'https://github.com/apache/airflo...,<!--\r\nThank you for contributing! Please mak...,"{'+1': 0, '-1': 0, 'confused': 0, 'eyes': 0, '...",https://api.github.com/repos/apache/airflow/is...,None
4,4,https://api.github.com/repos/apache/airflow/is...,https://api.github.com/repos/apache/airflow,https://api.github.com/repos/apache/airflow/is...,https://api.github.com/repos/apache/airflow/is...,https://api.github.com/repos/apache/airflow/is...,https://github.com/apache/airflow/pull/20763,1096844110,PR_kwDOAgUK284wr25c,20763,[WIP] Verify enough resources for breeze,{'avatar_url': 'https://avatars.githubusercont...,"[{'color': 'ededed', 'default': False, 'descri...",open,False,None,[],None,0,2022-01-08T05:57:03Z,2022-01-08T15:35:18Z,None,CONTRIBUTOR,None,False,{'diff_url': 'https://github.com/apache/airflo.

# Preprocessing

In [23]:
'''
There are lots of fields in the data. 
We will preprocess the data, engineer some features, and keep only the promising features.
'''
df.columns

Index(['index', 'url', 'repository_url', 'labels_url', 'comments_url',
       'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user',
       'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone',
       'comments', 'created_at', 'updated_at', 'closed_at',
       'author_association', 'active_lock_reason', 'draft', 'pull_request',
       'body', 'reactions', 'timeline_url', 'performed_via_github_app'],
      dtype='object')

In [24]:
df = df.drop(columns=['index', 'url', 'repository_url', 'labels_url', 'comments_url',
                      'events_url', 'html_url', 'performed_via_github_app', 'locked',
                      'active_lock_reason', 'assignees', 'milestone'])

# Processing timestamps
df['created_at'] = pd.to_datetime(df['created_at'], format="%Y/%m/%d")
df['closed_at'] = pd.to_datetime(df['closed_at'], format="%Y/%m/%d")

# Binarizing some features
df["has_pull_request"] = df.pull_request.apply(lambda x: True if x is not None else False)
df["has_comment"] = df.comments.apply(lambda x: True if x > 0 else False)
df['has_assignee'] = df['assignee'].apply(lambda x: True if x is not None else False)

# Extracting the lengths of text features `title` and `body`
df = df[df.body.notnull()]
df['length_of_title'] = df['title'].apply(lambda x: len(x))
df['length_of_body'] = df['body'].apply(lambda x: len(x))

# Removing issues created by bots
df['creator_user_login'] = df['user'].apply(lambda x: x.get('login'))
df = df[df['creator_user_login'] != 'dependabot[bot]']

In [25]:
df.head()

,id,node_id,number,title,user,labels,state,assignee,comments,created_at,updated_at,closed_at,author_association,draft,pull_request,body,reactions,timeline_url,has_pull_request,has_comment,has_assignee,length_of_title,length_of_body,creator_user_login
0,1096939225,PR_kwDOAgUK284wsGiW,20767,Rename amazon EMR hook name,{'avatar_url': 'https://avatars.githubusercont...,"[{'color': 'f89500', 'default': False, 'descri...",closed,None,3,2022-01-08 14:15:31+00:00,2022-01-08T15:26:11Z,2022-01-08 15:26:09+00:00,CONTRIBUTOR,False,{'diff_url': 'https://github.com/apache/airflo...,Renamed emr hook from Elastic MapReduce to Ama...,"{'+1': 0, '-1': 0, 'confused': 0, 'eyes': 0, '...",https://api.github.com/repos/apache/airflow/is...,True,True,False,27,1266,vinitpayal
1,1096938561,I_kwDOAgUK285BYfRB,20766,Status of testing Providers that were prepared...,{'avatar_url': 'https://avatars.githubusercont...,"[{'color': '5319e7', 'default': False, 'descri...",open,None,0,2022-01-08 14:12:48+00:00,2022-01-08T14:12:48Z,NaT,MEMBER,None,None,### Body\n\nI have a kind request for all the ...,"{'+1': 0, '-1': 0, 'confused': 0, 'eyes': 0, '...",https://api.github.com/repos/apache/airflow/is...,False,False,False,66,1033,potiuk
2,1096903757,PR_kwDOAgUK284wsBH7,20765,Add documentation for an ad-hoc release of 2 p...,{'avatar_url': 'https://avatars.githubusercont...,"[{'color': 'd4c5f9', 'default': False, 'descri...",closed,None,1,2022-01-08 11:24:41+00:00,2022-01-08T13:59:11Z,2022-01-08 13:59:11+00:00,MEMBER,False,{'diff_url': 'https://github.com/apache/airflo...,<!--\r\nThank you for contributing! Please mak...,"{'+1': 0, '-1': 0, 'confused': 0, 'eyes': 0, '...",https://api.github.com/repos/apache/airflow/is...,True,True,False,54,1072,potiuk
3,1096847036,PR_kwDOAgUK284wr3Zo,20764,Update metric name in documentation,{'avatar_url': 'https://avatars.githubusercont...,"[{'color': '0030b5', 'default': False, 'descri...",open,None,1,2022-01-08 06:15:11+00:00,2022-01-08T15:21:56Z,NaT,CONTRIBUTOR,False,{'diff_url': 'https://github.com/apache/airflo...,<!--\r\nThank you for contributing! Please mak...,"{'+1': 0, '-1': 0, 'confused': 0, 'eyes': 0, '...",https://api.github.com/repos/apache/airflow/is...,True,True,False,35,1185,humit0
4,1096844110,PR_kwDOAgUK284wr25c,20763,[WIP] Verify enough resources for breeze,{'avatar_url': 'https://avatars.githubusercont...,"[{'color': 'ededed', 'default': False, 'descri...",open,None,0,2022-01-08 05:57:03+00:00,2022-01-08T15:35:18Z,NaT,CONTRIBUTOR,False,{'diff_url': 'https://github.com/apache/airflo...,Check resources before installing docker: Clos...,"{'+1': 0, '-1': 0, 'confused': 0, 'eyes': 0, '...",https://api.github.com/repos/apache/airflow/is...,True,False,False,40,626,edithturn


In [ ]:
'''
Info on user submitting request
'''

In [33]:
def user_info(username):
    session = requests.Session()
    url = "https://api.github.com/users/" + username
    response = session.get(
        url,
        auth=(
            "AmirZahre", # your github username
            config.api_secret
        ),
    )
    return response.json()

In [49]:
user_info('AmirZahre')

{'login': 'AmirZahre',
 'id': 71795488,
 'node_id': 'MDQ6VXNlcjcxNzk1NDg4',
 'avatar_url': 'https://avatars.githubusercontent.com/u/71795488?v=4',
 'gravatar_id': '',
 'url': 'https://api.github.com/users/AmirZahre',
 'html_url': 'https://github.com/AmirZahre',
 'followers_url': 'https://api.github.com/users/AmirZahre/followers',
 'following_url': 'https://api.github.com/users/AmirZahre/following{/other_user}',
 'gists_url': 'https://api.github.com/users/AmirZahre/gists{/gist_id}',
 'starred_url': 'https://api.github.com/users/AmirZahre/starred{/owner}{/repo}',
 'subscriptions_url': 'https://api.github.com/users/AmirZahre/subscriptions',
 'organizations_url': 'https://api.github.com/users/AmirZahre/orgs',
 'repos_url': 'https://api.github.com/users/AmirZahre/repos',
 'events_url': 'https://api.github.com/users/AmirZahre/events{/privacy}',
 'received_events_url': 'https://api.github.com/users/AmirZahre/received_events',
 'type': 'User',
 'site_admin': False,
 'name': 'Amir',
 'company':

In [42]:
user_info('AmirZahre')['location']

'Canada'

In [32]:
# Parsing the `labels` to determine the number of labels and which of the popular labels are used for each issue
df['label_names'] = df['labels'].apply(lambda x: [str(i.get('name')) for i in x])
df['n_labels'] = df['label_names'].apply(lambda x: len(x))
not_labels = df.columns.to_list()

def label_column_to_true(row):
    for label in row.label_names:
        label = (
            label     
            .replace(":", "_")
            .replace(" ", "_")
            .replace("/", "_")
            .replace("'", "")
            .replace(".", "_")
            .replace("-", "_")
        )
        row[label] = 1
    return row


df_labels = df.apply(label_column_to_true, axis=1)
df_labels = df_labels.loc[:, ~df_labels.columns.isin(not_labels)]

# Labels that are used for at least 5% of issues
df_labels_keep = df_labels.T[(df_labels.isna().sum(axis=0) < len(df_labels)*0.95)].T
df_labels_keep.fillna(0, inplace=True)
df_labels_keep = df_labels_keep.astype("bool")
df_labels_keep = df_labels_keep.add_prefix('label_')
df = df.join(df_labels_keep)

In [33]:
# Parsing the issue creators to identify users who create issues frequently
df['creator_user_id'] = df['user'].apply(lambda x: x.get('id'))
creators = df.creator_user_id.value_counts()

# Issues created by top 0.05% issue creators
freq_creators = dict(df.creator_user_id.value_counts(normalize=True).loc[lambda x : x>0.0005]) 
df['frequent_issue_creator'] = df.creator_user_id.apply(lambda x: x in freq_creators.keys())
df = df.drop(columns = ['creator_user_id'])

In [34]:
# We now have our list of features to study
features = [
    'title',
    'body',
    'created_at',
    'state',
    'closed_at',
    'frequent_issue_creator',
    'author_association', 
    'length_of_title', 
    'length_of_body',
    'has_assignee', 
    'has_pull_request', 
    'has_comment',
    'comments',
    'n_labels'] + list(df_labels_keep.columns)

df.drop([c for c in df if c not in features], axis=1, inplace=True)

# Descriptive Analytics

In [35]:
''' 
Let's study the cleaned-up data to summarize their attributes and find interesting trends, if any. 

Does anything standout as an outlier, indicating it might be better to 
exclude certain records before we proceed to doing machine learning?
'''
df

,title,state,comments,created_at,closed_at,author_association,body,has_pull_request,has_comment,has_assignee,...,label_area_dev_tools,label_area_providers,label_area_webserver,label_full_tests_needed,label_kind_bug,label_kind_documentation,label_kind_feature,label_okay_to_merge,label_provider_Google,frequent_issue_creator
0,Rename amazon EMR hook name,closed,3,2022-01-08 14:15:31+00:00,2022-01-08 15:26:09+00:00,CONTRIBUTOR,Renamed emr hook from Elastic MapReduce to Ama...,True,True,False,...,False,True,False,False,False,False,False,True,False,False
1,Status of testing Providers that were prepared...,open,0,2022-01-08 14:12:48+00:00,NaT,MEMBER,### Body\n\nI have a kind request for all the ...,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,Add documentation for an ad-hoc release of 2 p...,closed,1,2022-01-08 11:24:41+00:00,2022-01-08 13:59:11+00:00,MEMBER,<!--\r\nThank you for contributing! Please mak...,True,True,False,...,False,True,False,False,False,True,False,True,False,True
3,Update metric name in documentation,open,1,2022-01-08 06:15:11+00:00,NaT,CONTRIBUTOR,<!--\r\nThank you for contributing! Please mak...,True,True,False,...,False,False,False,False,False,True,False,True,False,False
4,[WIP] Verify enough resources for breeze,open,0,2022-01-08 05:57:03+00:00,NaT,CONTRIBUTOR,Check resources before installing docker: Clos...,True,False,False,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19663,Misc spelling updates,closed,3,2015-06-05 17:50:46+00:00,2015-06-07 07:55:22+00:00,CONTRIBUTOR,I was reading over the code to understand how ...,True,True,False,...,False,False,False,False,False,False,False,False,False,False
19664,setup.py download URL,closed,3,2015-06-05 15:18:52+00:00,2015-06-05 17:24:57+00:00,CONTRIBUTOR,While installing I noticed the `url` and `down...,False,True,False,...,False,False,False,False,False,False,False,False,False,False
19665,Can't add external executors -> Plugin manager,closed,17,2015-06-05 14:28:38+00:00,2015-06-17 15:41:31+00:00,NONE,Would be nice to be able to add other executor...,False,True,False,...,False,False,False,False,False,False,False,False,False,False
19666,Bring in more resolution to hivestats,closed,0,2015-04-20 21:54:45+00:00,2015-04-20 22:58:48+00:00,MEMBER,,True,False,False,...,False,False,False,False,False,False,False,False,False,True


# Predictive Machine Learning

In [36]:
'''
Now that we understand our data better, let's see if we can predict 
whether or not an issue is closed given other attributes.
Should we treat all issues created over the years the same?

We notice that the classes are not balanced. Probably a good idea to address that.
'''
df.state.value_counts()

closed    18719
open        910
Name: state, dtype: int64

In [37]:
'''
Which of the features above should we include in our ML model?

The title and body are great text features which we could do some NLP on, but maybe later.
'''
def split_features_and_target(df, features_ml=None):
    target = df.state
    df.drop([c for c in df if c not in features_ml], axis=1, inplace=True)
    return df, target


features_ml, target = split_features_and_target(
    df,
    features_ml = [
    # List of features to use for ML
    ]
)

features_ml

""
0
1
2
3
4
...
19663
19664
19665
19666


In [38]:
# Split the data for training and testing


In [39]:
# Load in a model and train it


In [40]:
# Evaluate the model on the test data


In [41]:
# Plot the receiver operating characteristic curve and include the area under the curve
